![](https://findlogovector.com/wp-content/uploads/2019/03/vellore-institute-of-technology-vit-logo-vector.png)
#  <font color = BLUE> Bug Detection Using Neural Networks- An integrated ML Model
# <font color = RED>  **REVIEW- 3 - Software Engineering Project**
# <font color = GREEN> *SUBMITTED BY:*
#  <font color = PURPLE> *Vrushali Deshmukh (19BCE0033)*
#  <font color = PURPLE>  *Nimish K. Aggarwal (19BCE0014)*
#  <font color = PURPLE>  *Tarang Garg (19BCE0053)*
#  <font color = PURPLE>  *Shashwat Chaudhary (19BCE0056)*

In [5]:
# %% [code]
import pandas as pd
import glob
from sklearn import preprocessing


In [13]:
path = '../input/sw-project'

cnt = 5000

all_files = glob.glob(path + "/*.csv")

l = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    l.append(df)

frame = pd.concat(l, axis=0, ignore_index=True)
target = frame['bug']
frame = frame.drop(['bug'], axis = 1) 

x = frame.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)

train = pd.DataFrame(x_scaled)
    

In [14]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(train, target)


In [15]:
X_resampled = X_resampled.iloc[:cnt]
y_resampled = y_resampled.iloc[:cnt]

In [16]:
import numpy as np
imgs = []
for j in range(0,cnt):
    arr = []
    for i in range(0,20):
        arr.append(X_resampled[i][j])
    p = 8*8-20
    for i in range(0,p):
        arr.append(0)
    x = np.reshape(arr,(8,8))
    imgs.append(x)

y_new = []
for i in y_resampled:
    arr = [0]*8
    if i<8:
        arr[i]=1
    else:
        arr[0]=1
    y_new.append(arr)

In [18]:
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split


In [19]:
# %% [code]
X_train,x_test,Y_train,y_test = train_test_split(imgs , y_new , test_size = 0.1, random_state = 10)

# %% [code]
X_train = np.expand_dims(X_train,axis=3)
x_test = np.expand_dims(x_test,axis=3)

In [20]:
import keras
model = Sequential()
model.add(Conv2D(64, (1, 1), padding='same',input_shape=(8, 8, 1),activation='relu')) # Layer 1

def fire_module(squeeze,expand):
    model.add(Conv2D(squeeze, (1, 1), padding='same',activation='relu'))
    model.add(Conv2D(expand, (1, 1), padding='same',activation='relu'))
    model.add(Conv2D(expand, (3, 3), padding='same',activation='relu'))
    
fire_module(squeeze=16, expand=64)  # Layer 2
fire_module(squeeze=16, expand=64)  # Layer 3
fire_module(squeeze=32, expand=128) # Layer 4
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))

fire_module(squeeze=32, expand=128) # Layer 5
fire_module(squeeze=48, expand=192) # Layer 6
fire_module(squeeze=48, expand=192) # Layer 7
fire_module(squeeze=64, expand=256) # Layer 8
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))

fire_module(squeeze=64, expand=256) # Layer 9
model.add(Conv2D(64, (1, 1),padding='same',activation='relu')) # Layer 10

model.add(Flatten())

model.add(Dense(8, activation='softmax')) # Layer 11

learning_rate = 0.0005

opt = keras.optimizers.Adam(lr=learning_rate, decay=1e-6)

In [21]:
# %% [code]
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# %% [code]
model.fit(np.array(X_train),np.array(Y_train),batch_size=32,epochs=3,validation_data=(np.array(x_test),np.array(y_test)))

Epoch 1/3
141/141 [==============================] - 28s 184ms/step - loss: 0.6221 - accuracy: 0.9099 - val_loss: 0.3065 - val_accuracy: 0.9360
Epoch 2/3
141/141 [==============================] - 25s 178ms/step - loss: 0.2692 - accuracy: 0.9473 - val_loss: 0.2880 - val_accuracy: 0.9360
Epoch 3/3
141/141 [==============================] - 25s 180ms/step - loss: 0.2568 - accuracy: 0.9491 - val_loss: 0.2951 - val_accuracy: 0.9360


In [22]:
import sklearn
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score

temp = model.predict(np.array(x_test))

predicted_values = []
actual_values = []

for i in temp:
    val = 0
    for j in range(8):
        if i[j]:
            val = j
            break
    predicted_values.append(val)
    
for i in y_test:
    val = 0
    for j in range(8):
        if i[j]:
            val = j
            break
    actual_values.append(val)

model_data = []
accuracy_dict = {}
accuracy_dict["accuracy"] = sklearn.metrics.accuracy_score(predicted_values,actual_values)
accuracy_dict["mean_squared_error"] = sklearn.metrics.mean_squared_error(predicted_values,actual_values)
accuracy_dict["mean_absolute_error"] = sklearn.metrics.mean_absolute_error(predicted_values,actual_values)
model_data.append(accuracy_dict)

In [23]:
model_data

[{'accuracy': 0.936,
  'mean_squared_error': 0.134,
  'mean_absolute_error': 0.086}]